# Classify a Bank Customer and his Brand Royalty

# Load Tensor Flow

In [1]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(42)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
from tensorflow import keras

## 1. Read the dataset


In [4]:
df=pd.read_csv("bank.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [6]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


## 2.Drop the columns which are unique for all users like IDs

In [7]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [8]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [9]:
df['Gender']=le.fit_transform(df['Gender'])

In [10]:
df=pd.get_dummies(df,columns=['Geography'])

In [11]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [12]:
df.drop('Geography_France',axis=1,inplace=True)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,1


## 3. Distinguish the feature and target set

In [13]:
X = df.drop('Exited',axis=1)
Y = df['Exited']

## 4. Divide the data set into Train and test sets

In [14]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20,random_state=1)  
print(X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(8000, 11) (8000,)
(2000, 11) (2000,)


## 5. Normalize the train and test data 

In [15]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

## 6.Initialize &amp; build the model 

In [16]:
#Initialize Sequential model
model = tf.keras.models.Sequential()
#Add 1st hidden layer
model.add(tf.keras.layers.Dense(10, activation='relu',input_dim=11))
#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(10, activation='relu'))
#Add 3rd hidden layer
model.add(tf.keras.layers.Dense(10, activation='relu'))
#Add Output Layer
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

## Optimize the model

In [17]:
#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD()

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=32,epochs=30)

Train on 8000 samples, validate on 2000 samples
Epoch 1/30
8000/8000 [==============================] - 1s 115us/step - loss: 0.5660 - acc: 0.7572 - val_loss: 0.5105 - val_acc: 0.7925
Epoch 2/30
8000/8000 [==============================] - 1s 67us/step - loss: 0.4973 - acc: 0.7973 - val_loss: 0.4988 - val_acc: 0.7925
Epoch 3/30
8000/8000 [==============================] - 1s 66us/step - loss: 0.4884 - acc: 0.7973 - val_loss: 0.4914 - val_acc: 0.7925
Epoch 4/30
8000/8000 [==============================] - 1s 67us/step - loss: 0.4815 - acc: 0.7973 - val_loss: 0.4845 - val_acc: 0.7925
Epoch 5/30
8000/8000 [==============================] - 1s 67us/step - loss: 0.4752 - acc: 0.7973 - val_loss: 0.4777 - val_acc: 0.7925
Epoch 6/30
8000/8000 [==============================] - 1s 71us/step - loss: 0.4688 - acc: 0.7973 - val_loss: 0.4705 - val_acc: 0.7925
Epoch 7/30
8000/8000 [==============================] - 1s 67us/step - loss: 0.4620 - acc: 0.7973 - val_loss: 0.4637 - val_acc: 0.7925
Epoch 

In [19]:
#Review the model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 351
Trainable params: 351
Non-trainable params: 0
_________________________________________________________________


## Predict the results using 0.5 as a threshold

In [20]:
#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.5)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

## Print the Accuracy score and confusion matrix

In [21]:
scores = model.evaluate(X_test,y_test)
scores

2000/2000 [==============================] - 0s 80us/step


[0.3779260506629944, 0.8375]